In [7]:
import os
from dotenv import load_dotenv
from huggingface_hub import login
# .env 파일 로드 확인
load_dotenv('.env', verbose=True)

hf_token = os.getenv('HUGGINGFACEHUB_API_TOKEN')

login(hf_token)


In [4]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import json

# ✅ 문장 임베딩 모델 로드
embedding_model = SentenceTransformer("BM-K/KoSimCSE-roberta")

# ✅ 책 데이터 로드
with open("merged_books_filtered.json", "r", encoding="utf-8") as f:
    books_data = json.load(f)

# ✅ 책 요약을 임베딩 변환
book_summaries = [book["summary"] for book in books_data]
book_titles = [book["title"] for book in books_data]

# 🔹 임베딩 변환
book_embeddings = embedding_model.encode(book_summaries, convert_to_numpy=True)

# 🔹 L2 정규화 (코사인 유사도 적용)
faiss.normalize_L2(book_embeddings)  # 벡터 정규화

# 🔹 FAISS 인덱스 생성 (L2 정규화된 벡터 기반)
dimension = book_embeddings.shape[1]  # 벡터 차원 (768)
faiss_index = faiss.IndexFlatIP(dimension)  # 🚀 코사인 유사도 적용 (Inner Product)
faiss_index.add(book_embeddings)  # 벡터 추가

# ✅ FAISS 인덱스 저장
faiss.write_index(faiss_index, "book_faiss_cosine_index.bin")

print("✅ 코사인 유사도를 적용한 FAISS 인덱스가 저장되었습니다.")


No sentence-transformers model found with name BM-K/KoSimCSE-roberta. Creating a new one with mean pooling.


✅ 코사인 유사도를 적용한 FAISS 인덱스가 저장되었습니다.


In [6]:
import faiss
import json
import requests
from sentence_transformers import SentenceTransformer

# ✅ 1️⃣ FAISS 인덱스 로드
faiss_index = faiss.read_index("book_faiss_cosine_index.bin")

# ✅ 2️⃣ 책 데이터 로드
with open("merged_books_filtered.json", "r", encoding="utf-8") as f:
    books_data = json.load(f)

book_titles = [book["title"] for book in books_data]
book_summaries = [book["summary"] for book in books_data]

# ✅ 3️⃣ 문장 임베딩 모델 로드
embedding_model = SentenceTransformer("BM-K/KoSimCSE-roberta")

# ✅ 4️⃣ 네이버 API 호출 함수
def search_naver_api(user_query):
    """네이버 API를 사용하여 책 검색"""
    client_id = "YOUR_NAVER_CLIENT_ID"
    client_secret = "YOUR_NAVER_CLIENT_SECRET"
    url = "https://openapi.naver.com/v1/search/book.json"

    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    params = {
        "query": user_query,
        "display": 5,  # 최대 5권 검색
        "sort": "sim"
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()["items"]
    else:
        print(f"🚨 네이버 API 호출 실패: {response.status_code}")
        return []

# ✅ 5️⃣ FAISS 검색 함수
def find_similar_books(user_story, top_k=5):
    """유저 입력 스토리 → 코사인 유사도 기반 책 검색"""
    # 🔹 유저 입력을 문장 임베딩 변환
    user_embedding = embedding_model.encode([user_story], convert_to_numpy=True)
    faiss.normalize_L2(user_embedding)  # 벡터 정규화

    # 🔹 FAISS 검색
    distances, indices = faiss_index.search(user_embedding, top_k)

    # 🔹 검색된 책 목록 출력
    recommended_books = []
    for i in range(top_k):
        idx = indices[0][i]
        title = book_titles[idx]
        summary = book_summaries[idx]
        recommended_books.append(f"📖 **{title}**\n📌 {summary}\n")

    return recommended_books

# ✅ 6️⃣ FAISS + 네이버 API 하이브리드 검색 함수
def hybrid_search(user_story, top_k=5):
    """FAISS에서 먼저 검색하고, 부족하면 네이버 API 호출"""
    faiss_results = find_similar_books(user_story, top_k)

    # 🔹 검색 결과가 없거나, 유사한 책이 부족할 경우 → 네이버 API 검색
    if not faiss_results:
        print("⚠️ FAISS에서 유사한 책이 없음 → 네이버 API 검색 실행")
        naver_results = search_naver_api(user_story)

        if naver_results:
            print("✅ 네이버 API 검색 결과:")
            for book in naver_results:
                print(f"📖 **{book['title']}**\n📌 {book['description']}\n")

            # FAISS에 추가하기 (선택적)
            # add_to_faiss(naver_results)

        return naver_results
    else:
        return faiss_results

# ✅ 7️⃣ 테스트 실행
user_story = "뇌사한 딸을 포기하지 못하는 부모가 나오는 소설"
print(hybrid_search(user_story, top_k=3))


No sentence-transformers model found with name BM-K/KoSimCSE-roberta. Creating a new one with mean pooling.


['📖 **바람이 분다**\n📌 연인 세쓰코는 죽음을 앞두고 있다. 그러나 그들은 서로를 깊이 사랑했고, 오랫동안 서로의 곁에 머물고 싶었기에 끝내 사랑을 이루지 못한다. 이 소설은 사랑하는 연인의 애틋한 사랑 이야기와 연인을 떠나보낸 후 남겨진 이에게 진정한 삶은 무엇인지 되새긴다. \n', '📖 **가시고기**\n📌 가시고기 아빠 정호연은 백혈병과 골수이식이 아닌 더 이상 희망이 없는 상황 속에서, 아들을 살릴 수만 있다면 자신의 몸까지 내어주려는 아버지의 사랑을 그린다. 이 책은 아버지의 사랑이 어떤 모습일지 표현하지 않는다. 하지만 변할 수 없는 사랑은 변함없는 사랑으로 변함없는 감동을 전달한다.\n', '📖 **원도**\n📌 ‘나와 인간 사이 수명 중개인’의 이야기. 원작과 현실이 뒤섞인 이 소설은 삶과 죽음에 대한 질문을 던진다. 죽음을 앞두고 있는 소녀는 자신이 왜 죽지 않았는지 이유를 찾고, 그 이유를 밝혀낸다. 두 사람 모두 살인자처럼 보인다. 그러나 어느 순간, 소설가 최진영 유니버스의 시작이 된다.\n']
